In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import tensorflow as tf
import sys


In [2]:
#this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in the Shared Google Drive
FOLDERNAME = 'Shared drives/CS 230 Project'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
#import sys
sys.path.append('/content/drive/{}'.format(FOLDERNAME))

%cd /content/drive/$FOLDERNAME/

ModuleNotFoundError: No module named 'google.colab'

In [10]:
#datapath = "datasets/Gutenberg/"
datapath = ""
#train_df = pd.read_csv(datapath + "training.csv")
validation_df = pd.read_csv(datapath + "validation.csv").head(1000)
test_df = pd.read_csv(datapath + "test.csv")
test_df = test_df.head(17000)

In [11]:
def author_mapping(data):
    authors = []
    author_mappings = {}
    for index, row in data.iterrows():
        name = row['Author']
        if name in authors:
            continue
        authors.append(name)
        
    authors.sort()
    for i in range(len(authors)):
        author_mappings[authors[i]] = i
    return author_mappings 

In [12]:
author_mapping = author_mapping(test_df)
print(author_mapping)

{'Andrew Lang': 0, 'Anthony Trollope': 1, 'Bret Harte': 2, 'Charles Dickens': 3, 'Charles Kingsley': 4, 'Charlotte Mary Yonge': 5, 'Daniel Defoe': 6, 'Edgar Rice Burroughs': 7, 'Edward Phillips Oppenheim': 8, 'Edward Stratemeyer': 9, 'Frank Richard Stockton': 10, 'G K Chesterton': 11, 'George Alfred Henty': 12, 'George Bernard Shaw': 13, 'Hamlin Garland': 14, 'Harold Bindloss': 15, 'Henry James': 16, 'Henry Rider Haggard': 17, 'Herbert George Wells': 18, 'Jack London': 19, 'Jacob Abbott': 20, 'James Fenimore Cooper': 21, 'James Matthew Barrie': 22, 'James Otis': 23, 'Jerome Klapka Jerome': 24, 'John Galsworthy': 25, 'John Morley': 26, 'John Ruskin': 27, 'Joseph Conrad': 28, 'Louisa May Alcott': 29, 'Lyman Frank Baum': 30, 'Mark Twain': 31, 'Nathaniel Hawthorne': 32, 'Oscar Wilde': 33, 'P G Wodehouse': 34, 'R M Ballantyne': 35, 'Robert Louis Stevenson': 36, 'Rudyard Kipling': 37, 'Sir Arthur Conan Doyle': 38, 'Sir Walter Scott': 39, 'Thomas Carlyle': 40, 'Thomas Hardy': 41, 'Thomas Henr

In [13]:
count_vectorizer = feature_extraction.text.CountVectorizer()
#train_vectors = count_vectorizer.fit_transform(train_df['Text'])
test_vectors = count_vectorizer.fit_transform(test_df["Text"])

validation_vectors = count_vectorizer.transform(validation_df["Text"])

In [14]:
def extract_features(data):
  features = np.zeros((len(data.index), 2))
  for index, row in data.iterrows():
      text = row['Text']
      features[index, 0] = len(text) / 5
      words = text.split()
      features[index, 1] = len(text) / len(words)
  return features


In [15]:
def get_mappings(data, author_mapping):
    authors = data['Author']
    mappings = [author_mapping[author] for author in authors]
    data['mappings'] = mappings
    return data['mappings']

In [16]:
test_targets = get_mappings(test_df, author_mapping)
#test_vectors = extract_features(test_df)
#validation_vectors = extract_features(validation_df)
#train_targets = get_mappings(train_df, author_mapping)
validation_targets = get_mappings(validation_df, author_mapping)

In [17]:
num_rows = len(test_df.index)

In [18]:
from keras.utils import to_categorical
label_test = test_targets
label_v = validation_targets
# one hot encode
encoded_label_test = to_categorical(label_test)
encoded_label_v = to_categorical(label_v)



In [19]:
from keras.regularizers import l2

In [20]:
ann = tf.keras.models.Sequential()

In [21]:
#clf = linear_model.RidgeClassifier()
#scores = model_selection.cross_val_score(clf, train_vectors, train_targets, cv=3, scoring="f1")
#scores

In [22]:
ann.add(tf.keras.layers.Dense(units=100,  activation='relu'))

In [23]:
ann.add(tf.keras.layers.Dense(units=100, activation='relu'))

In [24]:
ann.add(tf.keras.layers.Dense(units=100, activation='relu'))

In [25]:
ann.add(tf.keras.layers.Dense(units=100, activation='relu'))

In [26]:
ann.add(tf.keras.layers.Dense(50, activation='softmax'))

In [27]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
ann.fit(test_vectors.todense(), encoded_label_test, batch_size = 32, epochs = 5)

In [ ]:
y_pred = ann.predict(validation_vectors.todense())
#y_pred = ???
y_pred

In [ ]:
y_output = np.zeros_like(y_pred)
y_output[np.arange(len(y_pred)), y_pred.argmax(1)] = 1

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
accuracy_score(encoded_label_v, y_output)